In [22]:
#importing libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
from matplotlib import pyplot
from scipy import stats
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from patsy import dmatrices
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import MinMaxScaler

In [23]:
# univariate lstm example
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [24]:
#Reading excel for ARIMA
df=pd.read_excel("Forecasting Assignment Data.xlsx","Sheet1",index_col=[0],parse_dates=True,squeeze=True)

In [25]:
# preparing independent and dependent features
def prepare_data(timeseries_data, n_features):
	X, y =[],[]
	for i in range(len(timeseries_data)):
		# find the end of this pattern
		end_ix = i + n_features
		# check if we are beyond the sequence
		if end_ix > len(timeseries_data)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [26]:
a=df.filter(['Gujarat'], axis=1)
# define input sequence
timeseries_data = np.array(a)
# choose a number of time steps
n_steps = 3
# split into samples
X, y = prepare_data(timeseries_data, n_steps)

In [27]:
X

array([[[319.5],
        [316.7],
        [301.9]],

       [[316.7],
        [301.9],
        [313.2]],

       [[301.9],
        [313.2],
        [320.7]],

       ...,

       [[366.9],
        [370.5],
        [377.4]],

       [[370.5],
        [377.4],
        [379.8]],

       [[377.4],
        [379.8],
        [377.2]]])

In [28]:
X.shape

(500, 3, 1)

In [29]:

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [30]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
16/16 [==============================] - 3s 8ms/step - loss: 85882.6875
Epoch 2/50
16/16 [==============================] - 0s 8ms/step - loss: 33885.2656
Epoch 3/50
16/16 [==============================] - 0s 7ms/step - loss: 2269.4504
Epoch 4/50
16/16 [==============================] - 0s 7ms/step - loss: 1155.7821
Epoch 5/50
16/16 [==============================] - 0s 7ms/step - loss: 997.8668
Epoch 6/50
16/16 [==============================] - 0s 7ms/step - loss: 945.1953
Epoch 7/50
16/16 [==============================] - 0s 7ms/step - loss: 956.3067
Epoch 8/50
16/16 [==============================] - 0s 7ms/step - loss: 798.4604
Epoch 9/50
16/16 [==============================] - 0s 7ms/step - loss: 764.2823
Epoch 10/50
16/16 [==============================] - 0s 9ms/step - loss: 748.3229
Epoch 11/50
16/16 [==============================] - 0s 8ms/step - loss: 691.8907
Epoch 12/50
16/16 [==============================] - 0s 7ms/step - loss: 710.1804
Epoch 13/50
16/16 [

In [31]:
from array import array

In [32]:
y_input = np.array(a[-3:])

In [33]:
a[-3:]

Gujarat
Dates              
2020-03-12    379.8
2020-04-12    377.2
2020-05-12    362.5

In [34]:
# demonstrate prediction for next 10 days
x_input = np.array([132.1,132.1,120.4])
temp_input=list(x_input)
lst_output=[]
i=0
while(i<90):
    
    if(len(temp_input)>3):
        x_input=np.array(temp_input[1:])
        print("{} month input {}".format(i,x_input))
        #print(x_input)
        x_input = x_input.reshape((1, n_steps, n_features))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} month output {}".format(i,yhat))
        temp_input.append(yhat[0][0])
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.append(yhat[0][0])
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.append(yhat[0][0])
        lst_output.append(yhat[0][0])
        i=i+1
    

print(lst_output)

[144.69453]
1 month input [132.1       120.4       144.6945343]
1 month output [[161.76453]]
2 month input [120.4        144.6945343  161.76452637]
2 month output [[165.27292]]
3 month input [144.69453 161.76453 165.27292]
3 month output [[177.85704]]
4 month input [161.76453 165.27292 177.85704]
4 month output [[191.58817]]
5 month input [165.27292 177.85704 191.58817]
5 month output [[201.13292]]
6 month input [177.85704 191.58817 201.13292]
6 month output [[210.02774]]
7 month input [191.58817 201.13292 210.02774]
7 month output [[218.85307]]
8 month input [201.13292 210.02774 218.85307]
8 month output [[226.7929]]
9 month input [210.02774 218.85307 226.7929 ]
9 month output [[234.0102]]
10 month input [218.85307 226.7929  234.0102 ]
10 month output [[240.73625]]
11 month input [226.7929  234.0102  240.73625]
11 month output [[246.91635]]
12 month input [234.0102  240.73625 246.91635]
12 month output [[252.53593]]
13 month input [240.73625 246.91635 252.53593]
13 month output [[257.

In [35]:
day_new=np.arange(1,10)
day_pred=np.arange(10,20)

In [36]:
len(timeseries_data)

503

In [37]:
len(lst_output)

90

In [38]:
day_new=np.arange(1,504)
day_pred=np.arange(504,594)

In [39]:
from matplotlib.pyplot import figure

In [40]:
figure(figsize=(12, 9), dpi=80)
plt.plot(day_new,timeseries_data)
plt.plot(day_pred,lst_output)